In [ ]:
from helper import *

import pandas as pd
import numpy as np
import os
import csv
import sys
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [8, 8]
# use LaTeX fonts in the plot
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})
%matplotlib inline

In [ ]:
EVAL_DIR = "/home/felix/todo/osm-tmp"
MLP_METHODS = ["kmeans", "merge", "gonzalez"]
MLP_LEVELS = [[int(2 ** i)] for i in np.arange(8.0, 13.5, 1.0)]
FAST_QUERY_METHODS = ["pcrp", "pch", "prp"]
QUERY_METHODS = ["normal", "bi"] + FAST_QUERY_METHODS
AREAS = ["saarland"]
print(MLP_METHODS, "with", MLP_LEVELS)

# eval

In [ ]:
df_graph = pd.DataFrame()
for area in AREAS:
    for mlp_method in MLP_METHODS:
        for level in MLP_LEVELS:
            for query in QUERY_METHODS:
                df_new = pd.read_json(EVAL_DIR + "/" + area + "-" + mlp_method + "-" +  "_".join(map(str, level)) + "-" + query + "-info.json", typ='series')
                df_new = pd.DataFrame([df_new])
                df_new["Area"] = area
                df_new["MLP_method"] = mlp_method
                df_new["Levels"] = "_".join(map(str, level))
                df_new["Query"] = query
                df_graph = pd.concat([df_graph, df_new], ignore_index=True)

In [ ]:
assert((df_graph.groupby(["amount_edges"]).size() == 5).all())

In [ ]:
df_graph

In [ ]:
fig, ax = plt.subplots()
speedups = list()
for area in AREAS:
    dijkstra = df_graph[(df_graph.Query == "normal") & (df_graph.Area == area)]["amount_used_edges"].iloc[0]
    bidijkstra = df_graph[(df_graph.Query == "bi") & (df_graph.Area == area)]["amount_used_edges"].iloc[0]
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            x = list()
            y = list()
            for partitions in MLP_LEVELS:
                tmp = df_graph[(df_graph.Area == area) & (df_graph.Query == query) & (df_graph.MLP_method == mlp) & (df_graph.Levels == "_".join(map(str, partitions)))]
                x.append(partitions[0])
                y.append(tmp["amount_used_edges"])
#                 speedups.append({"Query": query, "MLP": mlp, "_".join(map(str, partitions)): dijkstra / tmp["time"].mean()})
            plt.plot(x, y, marker=plot_get(query), color=plot_get(mlp), label=query + "-" + mlp, alpha=0.7)
plt.xlabel("MLP-Partition-Size")
plt.ylabel("edges per algorithm")
ax.set_yscale('log')
plt.legend(loc='upper left')
fig.savefig("used-edges-single-level.pgf", bbox_inches="tight")

In [ ]:
df_graph.groupby(["MLP_method", "Levels", "Query"])["amount_used_edges"].first()

In [ ]:
df_table = pd.DataFrame()
for area in AREAS:
    dijkstra = df_graph[(df_graph.Query == "normal") & (df_graph.Area == area)]["amount_used_edges"].iloc[0]
    print("original edge amount:", dijkstra)
    bidijkstra = df_graph[(df_graph.Query == "bi") & (df_graph.Area == area)]["amount_used_edges"].iloc[0]
    for query in FAST_QUERY_METHODS:
        for mlp in MLP_METHODS:
            line = dict()
            for partitions in MLP_LEVELS:
                tmp = df_graph[(df_graph.Area == area) & (df_graph.Query == query) & (df_graph.MLP_method == mlp) & (df_graph.Levels == "_".join(map(str, partitions)))]
                line[partitions[0]] = tmp["amount_used_edges"].values[0]
            df_new = pd.DataFrame([line])
            df_new["MLP_method"] = mlp
            df_new["Query"] = query
            df_table = pd.concat([df_table, df_new], ignore_index=True)

In [ ]:
df_edges = df_table.groupby(["Query", "MLP_method"]).first()
latex = df_edges.to_latex(float_format="{:0.1f}".format)
df_edges

In [ ]:
# fixup ugly latex code to have single line header
latex_list = latex.splitlines()

latex_list[0] = latex_list[0].replace('ll', 'll|', 1)

partitions = latex_list[2]
headers = latex_list[3].split("&")
headers = [headers[0], headers[1], " \multicolumn{" + str(len(headers) - 2) + "}{c}{amount of partitions} \\\\"]

latex_list[3] = partitions
latex_list[2] = "&".join(headers)

latex_list.insert(len(latex_list)-8, '\midrule')
latex_list.insert(len(latex_list)-5, '\midrule')
latex_new = '\n'.join(latex_list)
with open("edges.tex", "w") as latex_file:
    latex_file.writelines(latex_new)